In [1]:
!pip install langchain huggingface_hub tiktoken Chromadb pypdf sentence-transformers torch accelerate docx2txt
!pip install llama-cpp-python

      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.18-cp310-cp310-manylinux_2_35_x86_64.whl size=1962453 sha256=05a68fcba684040c75e41f6687d68cd32

In [2]:
# !huggingface-cli download TheBloke/zephyr-7B-alpha-GGUF zephyr-7b-alpha.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False



!huggingface-cli download TheBloke/Barcenas-Mistral-7B-GGUF barcenas-mistral-7b.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False


# !huggingface-cli download TheBloke/Llama-2-7B-Chat-GGUF llama-2-7b-chat.Q5_K_M.gguf --local-dir . --local-dir-use-symlinks False
# !huggingface-cli download TheBloke/Mistral-7B-OpenOrca-oasst_top1_2023-08-25-v1-GGUF mistral-7b-openorca-oasst_top1_2023-08-25-v1.Q5_K_M.gguf --local-dir . --local-dir-use-symlinks False
# !huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GGUF mistral-7b-instruct-v0.1.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False



downloading https://huggingface.co/TheBloke/Barcenas-Mistral-7B-GGUF/resolve/main/barcenas-mistral-7b.Q4_K_M.gguf to /root/.cache/huggingface/hub/tmpmul2c_sa
Storing https://huggingface.co/TheBloke/Barcenas-Mistral-7B-GGUF/resolve/main/barcenas-mistral-7b.Q4_K_M.gguf in local_dir at ./barcenas-mistral-7b.Q4_K_M.gguf (not cached).
./barcenas-mistral-7b.Q4_K_M.gguf


In [3]:
# Definición de la localización del modelo.\
MODEL_PATH = "barcenas-mistral-7b.Q4_K_M.gguf"

In [4]:
import tempfile
import os
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

prompt_template_questions = """Eres un experto en crear preguntas 
de práctica basadas en material de dominio general. 
Tu objetivo es escribir preguntas y respuestas a partir 
de un contexto dado. Lo haces haciendo preguntas sobre 
el texto a continuación:
{text}
Crea preguntas que prepararán a una persona para responderlas. 
Asegúrate de no perder ninguna información importante.

PREGUNTAS:"""

refine_template_questions = """ Eres un experto en crear 
preguntas de práctica basadas en material de estudio.
 Tu objetivo es ayudar a una persona a prepararse para
  responder estas preguntas. 
 Hemos recibido algunas preguntas de práctica hasta 
 cierto punto: {existing_answer}. 
 Tenemos la opción de refinar las preguntas existentes o 
 agregar nuevas. (solo si es necesario) 
 con algo más de contexto a continuación.
{text}
Dado el nuevo contexto, refina las preguntas originales en 
español. Si el contexto no es útil, por favor proporciona 
las preguntas originales.

PREGUNTAS: """

PROMPT_QUESTIONS = PromptTemplate(template=prompt_template_questions, input_variables=["text"])

REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template_questions,
)


In [5]:
# Se crea el reader para cargar el archivo, en este caso un pdf que debe subir al colab

loader = PyPDFLoader("./example.pdf")
data = loader.load()

In [ ]:
print(data)

In [ ]:

text_question_gen = ''
for page in data:
   text_question_gen += page.page_content

text_splitter_question_gen = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=50)

text_chunks_question_gen = text_splitter_question_gen.split_text(text_question_gen)

docs_question_gen = [Document(page_content=t) for t in text_chunks_question_gen]

llm_question_gen = LlamaCpp(model_path=MODEL_PATH,temperature=0.75,top_p=1,verbose=True, n_ctx=4096)

question_gen_chain = load_summarize_chain(llm=llm_question_gen, chain_type="refine", verbose=True,
                                question_prompt=PROMPT_QUESTIONS, refine_prompt=REFINE_PROMPT_QUESTIONS)

questions = question_gen_chain.run(docs_question_gen)

embeddings = HuggingFaceEmbeddings(model_name="jaimevera1107/all-MiniLM-L6-v2-similarity-es", model_kwargs={"device": "cpu"})

vector_store = Chroma.from_documents(docs_question_gen, embeddings)


llm_answer_gen = LlamaCpp(model_path=MODEL_PATH,temperature=0.75,top_p=1,verbose=True,n_ctx=4096)

answer_gen_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen, chain_type="stuff",
                                                retriever=vector_store.as_retriever(k=2))

question_list = questions.split("\n")

answers_dir = os.path.join(tempfile.gettempdir(), "answers")
os.makedirs(answers_dir, exist_ok=True)

qa_file_path = os.path.join(answers_dir, "questions_and_answers.txt")

with open(qa_file_path, "w") as qa_file:
        # Answer each question and save to the file
        for idx, question in enumerate(question_list):
            answer = answer_gen_chain.run(question)
            print(f"Question: {question}\nAnswer: {answer}")
            qa_file.write(f"Question {idx + 1}: {question}\n")
            qa_file.write(f"Answer {idx + 1}: {answer}\n")
            qa_file.write("--------------------------------------------------\n\n")



AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Eres un experto en crear preguntas de práctica basadas en material de dominio general. Tu objetivo es escribir preguntas y respuestas a partir de un contexto dado. Lo haces haciendo preguntas sobre el texto a continuación:

La materia oscura es una sustancia invisible que permea el espacio y conforma
aproximadamente el 27% de la composición del universo. Esta materia oscura interactúa
con la gravedad, pero no puede ser vista directamente ya que sus partículas son
incapaces de producir, reflejar o absorber luz
La composición exacta de la materia oscura es un misterio. Se han propuesto varias
categorías de materia oscura, incluyendo:
Materia oscura caliente: partículas que viajan con velocidades relativistas. Los neutrinos
son un ejemplo de materia oscura caliente, pero los límites de los neutrinos indican que
los neutrinos ordinarios solo harían una pequeña contribución a la densid

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


...

Llama.generate: prefix-match hit


Question: 3. ¿Cómo interactúa con la gravedad la materia oscura?
Answer:  La materia oscura interacciona con la gravidad a través de fuerzas gravitatorias. Esto significa que, como cualquier otra forma de energía o masa, la materia oscura tiene una fuerza de gravedad que es proporcional a su masa. Sin embargo, debido a su naturaleza inobservable y desconocida, la relación entre la masa de la materia oscura y su fuerza gravitatoria aún no está completamente clarificada. En general, se cree que la presencia de la materia oscura en el universo es una de las razones principales para explicar por qué el universo se expande a una velocidad acelerada y no colapsa en sí mismo, como se esperaba en teorías anteriores.


Llama.generate: prefix-match hit


Question: 4. ¿Es visible la materia oscura directamente?
Answer:  No, la materia oscura no es visible directamente ya que sus partículas son incapaces de producir, reflejar o absorber luz. Sin embargo, su presencia puede ser detectada a través de su efecto gravitacional y la distribución de la materia en el universo.


Llama.generate: prefix-match hit


# Nota Importante
Debe subir un archivo pdf a colab y pasarselo a este codigo
```python
loader = PyPDFLoader("./example.pdf")
```
para que le funcione correctamente.

## Side notes
1. El modelo barcenas esta entrenado con un dataset en español por ende es mejor en este idioma, no obstante al ser un fine tuning de el modelo mistral se puede usar en ese idioma tambien

# Referencias
[Repositorio de referencia](https://github.com/InsightEdge01/SmartExamApp/tree/master)
